In [ ]:
import os
import numpy as np
import pandas as pd

def load_eeg_data(data_dir):
    rows = []

    for condition in ['Healthy', 'AD']:
        for state in ['Eyes_open', 'Eyes_closed']:
            folder_path = os.path.join(data_dir, condition, state)
            if not os.path.exists(folder_path):
                print(f"Directory {folder_path} not found.")
                continue

            for paciente in os.listdir(folder_path):
                paciente_folder = os.path.join(folder_path, paciente)
                if os.path.isdir(paciente_folder):
                    for file in os.listdir(paciente_folder):
                        if file.endswith('.txt'):
                            file_path = os.path.join(paciente_folder, file)
                            try:
                                data = np.loadtxt(file_path)
                                rows.append({
                                    'Condition': condition,
                                    'State': state,
                                    'Patient': paciente,
                                    'File': file,
                                    'Signal': data
                                })
                            except Exception as e:
                                print(f"Error loading {file_path}: {e}")
    
    eeg_df = pd.DataFrame(rows)
    print("EEG data loaded successfully. Number of records:", len(eeg_df))
    return eeg_df

In [ ]:
# Étape 1 : Charger les données EEG
data_dir = "./EEG_data"
eeg_data_df = load_eeg_data(data_dir)

# Étape 2 : Nettoyage des données
def clean_data(df):
    # Supprimer les lignes avec des valeurs manquantes dans le signal
    df = df.dropna(subset=['Signal'])
    print("Valeurs manquantes supprimées. Nombre de lignes restantes :", len(df))

    # Filtrer les valeurs aberrantes : par exemple, en supprimant les signaux avec des amplitudes extrêmes
    df = df[df['Signal'].apply(lambda x: np.all(np.abs(x) < 1000))]  # Exemple de seuil
    print("Valeurs aberrantes filtrées. Nombre de lignes restantes :", len(df))

    # Normaliser les données si nécessaire
    df['Signal'] = df['Signal'].apply(lambda x: (x - np.mean(x)) / np.std(x))
    print("Les signaux ont été normalisés.")

    return df

# Appliquer le nettoyage des données
eeg_data_cleaned = clean_data(eeg_data_df)


In [ ]:
# Afficher les données pour les premiers patients
print(eeg_data_df.head())


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

# Charger les données brutes EEG (supposons que eeg_data_cleaned soit votre DataFrame contenant les signaux bruts)
# Nous allons utiliser la colonne 'Signal' comme données d'entrée

# Préparation des données
def prepare_raw_data(df):
    # Les signaux EEG bruts sont des séries de valeurs. Pour simplifier, nous allons les aplatir.
    X = np.vstack(df['Signal'].values)  # Transformer la colonne 'Signal' en matrice d'entrée
    y = df['Condition'].apply(lambda x: 1 if x == 'AD' else 0)  # 1 pour AD, 0 pour Healthy
    return X, y

# Appliquer la préparation
X, y = prepare_raw_data(eeg_data_cleaned)

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Créer et entraîner le modèle d'arbre de décision
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)
print("Modèle d'arbre de décision entraîné avec succès.")

# Prédire les résultats sur l'ensemble de test
y_pred = model.predict(X_test)

# Afficher le rapport de classification et la précision
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

# Affichage de l'arbre de décision
from sklearn.tree import export_text
tree_text = export_text(model)
print("Arbre de décision :\n", tree_text)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

# Afficher l'arbre de décision
plt.figure(figsize=(20, 10))  # Ajuster la taille de l'image
plot_tree(
    model,
    feature_names=[f'Feature {i}' for i in range(X.shape[1])],  # Utilisation de noms génériques pour les caractéristiques
    class_names=['Healthy', 'AD'],
    filled=True,
    rounded=True,
    fontsize=10
)
plt.title("Arbre de décision - EEG Data")
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

# Afficher le rapport de classification
print("Rapport de classification - Arbre de Décision :\n", classification_report(y_test, y_pred))

# Calculer l'accuracy sur les données de test
accuracy = accuracy_score(y_test, y_pred)

# Afficher l'accuracy
print(f"Accuracy du modèle : {accuracy:.2f}")


from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Calculer la matrice de confusion
cm_decision_tree = confusion_matrix(y_test, y_pred)

# Afficher la matrice de confusion
plt.figure(figsize=(10, 7))
sns.heatmap(cm_decision_tree, annot=True, fmt="d", cmap="Greens", xticklabels=['Healthy', 'AD'], yticklabels=['Healthy', 'AD'])
plt.xlabel("Prédiction")
plt.ylabel("Vraie Classe")
plt.title("Matrice de Confusion - Arbre de Décision")
plt.show()



In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Créer et entraîner le modèle de forêt aléatoire
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_model.fit(X_train, y_train)
print("Modèle de forêt aléatoire entraîné avec succès.")

# Prédire les résultats sur l'ensemble de test
y_pred_rf = random_forest_model.predict(X_test)

import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

# Sélectionner un arbre de la forêt aléatoire (par exemple, le premier arbre)
plt.figure(figsize=(20, 10))  # Ajuster la taille de l'image
plot_tree(
    random_forest_model.estimators_[0],  # Premier arbre de la forêt
    feature_names=[f'Feature {i}' for i in range(X.shape[1])],  # Utiliser les noms de caractéristiques génériques
    class_names=['Healthy', 'AD'],
    filled=True,
    rounded=True,
    fontsize=10
)
plt.title("Affichage de l'arbre - Forêt Aléatoire")
plt.show()


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Afficher le rapport de classification
print("Rapport de classification - Forêt Aléatoire :\n", classification_report(y_test, y_pred_rf))
print("Accuracy (précision) de la Forêt Aléatoire :", accuracy_score(y_test, y_pred_rf))

from sklearn.metrics import confusion_matrix
import seaborn as sns

# Calculer la matrice de confusion
cm = confusion_matrix(y_test, y_pred_rf)

# Afficher la matrice de confusion sous forme graphique
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Healthy', 'AD'], yticklabels=['Healthy', 'AD'])
plt.xlabel("Prédiction")
plt.ylabel("Vraie Classe")
plt.title("Matrice de Confusion - Forêt Aléatoire")
plt.show()



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Étape 1 : Créer et entraîner le modèle KNN
knn_model = KNeighborsClassifier(n_neighbors=5)  # Nombre de voisins ajustable
knn_model.fit(X_train, y_train)
print("Modèle KNN entraîné avec succès.")

# Étape 2 : Prédire les résultats sur l'ensemble de test
y_pred_knn = knn_model.predict(X_test)

# Étape 3 : Afficher les premières prédictions et les étiquettes réelles correspondantes
print("Prédictions KNN sur les premières données de test :")
for i in range(5):  # Afficher les 5 premières prédictions
    print(f"Prédiction : {y_pred_knn[i]}, Véritable étiquette : {y_test.iloc[i]}")

# Étape 4 : Calculer et afficher l'accuracy
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"Accuracy du modèle KNN : {accuracy_knn:.2f}")

# Étape 5 : Afficher le rapport de classification
print("Rapport de classification - KNN :\n", classification_report(y_test, y_pred_knn))

# Étape 6 : Matrice de Confusion
cm_knn = confusion_matrix(y_test, y_pred_knn)

# Afficher la matrice de confusion pour KNN
plt.figure(figsize=(10, 7))
sns.heatmap(cm_knn, annot=True, fmt="d", cmap="Blues", xticklabels=['Healthy', 'AD'], yticklabels=['Healthy', 'AD'])
plt.xlabel("Prédiction")
plt.ylabel("Vraie Classe")
plt.title("Matrice de Confusion - KNN")
plt.show()


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Étape 1 : Créer et entraîner le modèle SVM
svm_model = SVC(kernel='linear', random_state=42)  # Utilisation d'un noyau linéaire
svm_model.fit(X_train, y_train)
print("Modèle SVM entraîné avec succès.")

# Étape 2 : Prédire les résultats sur l'ensemble de test
y_pred_svm = svm_model.predict(X_test)

# Étape 3 : Afficher les premières prédictions et les étiquettes réelles correspondantes
print("Prédictions SVM sur les premières données de test :")
for i in range(5):  # Afficher les 5 premières prédictions
    print(f"Prédiction : {y_pred_svm[i]}, Véritable étiquette : {y_test.iloc[i]}")

# Étape 4 : Calculer et afficher l'accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"Accuracy du modèle SVM : {accuracy_svm:.2f}")

# Étape 5 : Afficher le rapport de classification
print("Rapport de classification - SVM :\n", classification_report(y_test, y_pred_svm))

# Étape 6 : Matrice de Confusion
cm_svm = confusion_matrix(y_test, y_pred_svm)

# Afficher la matrice de confusion pour SVM
plt.figure(figsize=(10, 7))
sns.heatmap(cm_svm, annot=True, fmt="d", cmap="Greens", xticklabels=['Healthy', 'AD'], yticklabels=['Healthy', 'AD'])
plt.xlabel("Prédiction")
plt.ylabel("Vraie Classe")
plt.title("Matrice de Confusion - SVM")
plt.show()
